# Wildfire Prediction

In [4]:
import numpy as np
import pandas as pd
df = pd.read_csv("modis_2021_Canada.csv")

df.head()

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type
0,54.8753,-117.4533,321.9,1.1,1.0,2021-01-01,522,Terra,MODIS,91,6.03,266.1,0.0,N,0
1,54.2008,-118.9019,313.9,1.2,1.1,2021-01-01,522,Terra,MODIS,88,6.03,267.7,23.4,N,0
2,55.3087,-111.7090,313.8,2.4,1.5,2021-01-02,1934,Terra,MODIS,0,6.03,265.3,67.6,D,0
3,54.4509,-107.1336,305.0,4.0,1.9,2021-01-02,1934,Terra,MODIS,34,6.03,265.1,94.0,D,0
4,54.4552,-107.1247,322.5,4.0,1.9,2021-01-02,1934,Terra,MODIS,0,6.03,265.8,197.2,D,0


### Get the dataset

### Prepare the dataset

### Do stuff